In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [4]:
from utils.preprocessing import read_excel_sheets
table_names, tables = read_excel_sheets('./data/data.xls')